In [24]:
import argparse, time
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from dgl import DGLGraph
from dgl import graph_index
from dgl.graph_index import disjoint_partition
from dgl.data import register_data_args, load_data
import math

In [37]:
    parser = argparse.ArgumentParser(description='GCN')
    parser.add_argument("--dataset", type=str, default="reddit-self-loop",
            help="dropout probability")
    parser.add_argument("--dropout", type=float, default=0.5,
            help="dropout probability")
    parser.add_argument("--gpu", type=int, default=-1,
            help="gpu")
    parser.add_argument("--lr", type=float, default=1e-2,
            help="learning rate")
    parser.add_argument("--n-epochs", type=int, default=200,
            help="number of training epochs")
    parser.add_argument("--n-hidden", type=int, default=16,
            help="number of hidden gcn units")
    parser.add_argument("--n-layers", type=int, default=1,
            help="number of hidden gcn layers")
    parser.add_argument("--weight-decay", type=float, default=5e-4,
            help="Weight for L2 loss")
    parser.add_argument("--self-loop", action='store_true',
            help="graph self-loop (default=False)")
    parser.set_defaults(self_loop=False)
    args = parser.parse_args(args=[])

In [38]:
    data = load_data(args)
    features = torch.FloatTensor(data.features)
    labels = torch.LongTensor(data.labels)
    train_mask = torch.ByteTensor(data.train_mask)
    val_mask = torch.ByteTensor(data.val_mask)
    test_mask = torch.ByteTensor(data.test_mask)
    in_feats = features.shape[1]
    n_classes = data.num_labels
    n_edges = data.graph.number_of_edges()

Finished data loading.
  NumNodes: 232965
  NumEdges: 114848857
  NumFeats: 602
  NumClasses: 41
  NumTrainingSamples: 153431
  NumValidationSamples: 23831
  NumTestSamples: 55703


In [39]:
g = DGLGraph(data.graph)

In [40]:
partition_size = 32

In [41]:
Edge = []
Edge_number = []
partition_number = math.ceil(g.number_of_nodes() / partition_size)
print("the graph split to {:d} part". format(partition_number))
for node_id in range(partition_number):
    #print(node_id)
    if node_id == partition_number-1:
        index = list(range(partition_size*node_id,g.number_of_nodes()))
    else:
        index = list(range(partition_size*node_id,partition_size*(node_id+1)))
    src, dst = g.out_edges(index)
    Edge.append(list(zip(src.tolist(),dst.tolist())))
    Edge_number.append(src.shape[0])

the graph split to 7281 part


In [42]:
max(Edge_number)

42368

In [21]:
n_edges

10556

In [23]:
Edge_number

[127,
 129,
 194,
 134,
 164,
 289,
 134,
 132,
 118,
 151,
 161,
 134,
 115,
 221,
 171,
 128,
 149,
 330,
 166,
 213,
 124,
 142,
 110,
 231,
 130,
 119,
 118,
 104,
 122,
 91,
 130,
 130,
 114,
 122,
 118,
 144,
 125,
 116,
 140,
 149,
 140,
 154,
 118,
 91,
 102,
 126,
 117,
 169,
 118,
 139,
 103,
 115,
 105,
 137,
 87,
 86,
 103,
 94,
 86,
 107,
 86,
 90,
 77,
 93,
 84,
 92,
 72,
 121,
 109,
 108,
 81,
 93,
 105,
 107,
 94,
 133,
 82,
 88,
 112,
 81,
 104,
 85,
 69,
 100,
 54]

In [16]:
Edge_number

124

In [36]:
index = list(range(1,2))

TypeError: 'list' object is not callable